# Domain Retriever for MLflow

In [ ]:
# !pip install -q requirements.txt

In [1]:
class S3_provider():
    """
    Класс для взаимодействия с хранилищем S3.

    Этот класс предоставляет методы для загрузки файлов из S3 хранилища. Он используется для загрузки и
    хранения моделей и данных, необходимых для работы сервиса.
    """
    
    def __init__(self):
        """
        Инициализация провайдера S3.

        Настраивает соединение с хранилищем S3, используя заданные параметры подключения.
        """
        # Работа с облачными сервисами
        import s3fs
        import boto3
        from botocore.client import Config
    
        # Настройки MinIO
        minio_access_key  = "minio_access_key"
        minio_secret_key  = "minio_secret_key"
        minio_endpoint    = "minio_endpoint"
        minio_bucket_name = "minio_bucket_name"

        self.s3 = boto3.resource('s3',
                            endpoint_url=minio_endpoint,
                            aws_access_key_id='minio_access_key',
                            aws_secret_access_key='minio_secret_key',
                            config=Config(signature_version='s3v4'),
                            region_name='us-east-1')

        self.bucket_name = 'prod-aiplatform-data'
        self.bucket = self.s3.Bucket(self.bucket_name)

        self.s3 = s3fs.S3FileSystem(anon=False, 
                            key=minio_access_key, 
                            secret=minio_secret_key, 
                            client_kwargs={"endpoint_url": minio_endpoint},
                            use_ssl=False)


    def download_from_s3(self, s3_folder: str, local_folder: str) -> str:
        """
        Загрузка файлов из S3 хранилища в локальную директорию.

        Description:
            Метод автоматически загружает все файлы из указанной папки в S3 хранилище в локальную директорию.
            Если локальная директория не существует, она будет создана вместе с необходимыми поддиректориями.
            Процесс загрузки логируется, предоставляя информацию о статусе загрузки каждого файла.
            В случае возникновения ошибки в процессе загрузки, метод логирует ошибку и возвращает `None`.
        Args:
            s3_folder (str): Путь к папке в S3 хранилище. Указывается от корня бакета.
            local_folder (str): Путь к локальной папке для сохранения файлов.
        Returns:
            str or None: Возвращает путь к локальной директории, куда были загружены файлы, если процесс завершился
                         успешно. Возвращает `None`, если в процессе загрузки произошла ошибка.
        Exceptions:
            Логирует исключения, связанные с ошибками доступа к S3 или невозможностью создать локальные директории.
        """
        import os
        import logging
        
        logging.basicConfig(
            level=logging.INFO,
            format="%(asctime)s - [%(levelname)s]: %(message)s",
            handlers=[
                logging.handlers.RotatingFileHandler(
                    filename="log.log",
                    mode="a",
                    maxBytes=1024,
                    backupCount=1,
                    encoding=None,
                    delay=0),
                logging.StreamHandler()
                ]
              )

        if not os.path.exists(local_folder):
            try:
                for obj in self.bucket.objects.filter(Prefix=s3_folder):
                    # Формирование пути для сохранения файла локально
                    local_path = os.path.join(local_folder, os.path.basename(obj.key))

                    # Создание локальной папки, если она не существует
                    os.makedirs(os.path.dirname(local_path), exist_ok=True)

                    # Загрузка файла из S3 в локальную папку
                    self.bucket.download_file(obj.key, local_path)

                logging.info(f"Файлы успешно загружены из S3 в {local_path}")
            except Exception as e:
                logging.info(f"Ошибка при загрузке файла из S3: {str(e)}")
                return None
        return local_folder

In [5]:
import mlflow
import boto3

class Domain_Retriever(mlflow.pyfunc.PythonModel):
    """
    Классификатор для обработки естественного языка (NLU), специализирующийся на поиске и классификации терминов и запросов.
    """
    import  pandas as pd
    import torch
    
    def __init__(self, file_path = './Question.docx'):
        """
        Инициализирует NLU_Classifier с заданной моделью NLU.

        Args:
            nlu_model (str): Путь к предобученной модели NLU.
        """
        self.synonyms_dicts = [['ПАК ЗВП', 'PTAF', 'WAF', 'ПТАФ', 'ВАФ', 'программно-аппаратный комплекс защиты веб приложений', 'веб-файервол', 'web firewall'],
                                 ['Антивирус', 'KSC', 'Касперский', 'Kaspersky', 'антивирус касперского', 'антивирусная защита', 'АВЗ', 'комплексная система антивирусной защиты', 'КСАЗ'],
                                 ['NGate', 'шифрование по ГОСТ', 'ГОСТ шифрование', 'НГейт', 'TLS-ГОСТ'],
                                 ['Hashicorp Vault', 'Vault', 'Вольт', 'Хашикорп вольт', 'менеджер секретов', 'система хранения секретов', 'система управления секретами'],
                                 ['КриптоПро', 'CryptoPro', 'CSP', 'криптопровайдер'],
                                 ['СТП', 'ТП', 'техподдержка', 'техническая поддержка', 'служба технической поддержки'],
                                 ['Центр обеспечения безопасности', 'Центр обеспечения информационной безопасности', 'Security Operations Center', 'SOC', 'СОК'],
                                 ['NTP', 'сервис точного времени', 'сервис времени', 'Network Time Protocol'],
                                 ['СЗИ', 'система защиты информации'],
                                 ['СрЗИ', 'средство защиты информации'],
                                 ['СКЗИ', 'средство криптозащиты', 'средство криптографической защиты'],
                                 ['ЭП', 'электронная подпись']
                                 ]

        self.aditional_data = ['ПАК ЗВП это программно-аппаратный комплекс защиты веб приложений производства Positive Technologies. Подробнее про это можно прочитать тут: https://www.ptsecurity.com/ru-ru/products/af/',
                              ' PTAF это Positive Technologies Application Firewall — межсетевой экрана уровня веб-приложений (web application firewall, WAF)¹, предназначенного для защиты веб-ресурсов от атак из списка OWASP Top 10, DDoS-атак уровня приложений, а также зловредных ботов.  Подробнее про это можно прочитать тут: https://www.ptsecurity.com/ru-ru/products/af/',
                              ' WAF это межсетевой экрана уровня веб-приложений (web application firewall, WAF)¹, предназначенного для защиты веб-ресурсов от атак из списка OWASP Top 10, DDoS-атак уровня приложений, а также зловредных ботов, часто называется Positive Technologies Application Firewall (PTAF).  Подробнее про это можно прочитать тут: https://www.ptsecurity.com/ru-ru/products/af/',
                              ' ПТАФ это Positive Technologies Application Firewall — межсетевой экрана уровня веб-приложений (web application firewall, WAF)¹, предназначенного для защиты веб-ресурсов от атак из списка OWASP Top 10, DDoS-атак уровня приложений, а также зловредных ботов.  Подробнее про это можно прочитать тут: https://www.ptsecurity.com/ru-ru/products/af/',
                              ' ВАФ это межсетевой экрана уровня веб-приложений (web application firewall, WAF)¹, предназначенного для защиты веб-ресурсов от атак из списка OWASP Top 10, DDoS-атак уровня приложений, а также зловредных ботов, часто называется Positive Technologies Application Firewall (PTAF).  Подробнее про это можно прочитать тут: https://www.ptsecurity.com/ru-ru/products/af/',
                              ' программно-аппаратный комплекс защиты веб приложений это программно-аппаратный комплекс производства Positive Technologies для защиты веб приложений от атак из списка OWASP Top 10, DDoS-атак уровня приложений, а также зловредных ботов.  Подробнее про это можно прочитать тут: https://www.ptsecurity.com/ru-ru/products/af/',
                              ' веб-файервол это межсетевой экрана уровня веб-приложений (web application firewall, WAF)¹, предназначенного для защиты веб-ресурсов от атак из списка OWASP Top 10, DDoS-атак уровня приложений, а также зловредных ботов, часто называется Positive Technologies Application Firewall (PTAF).  Подробнее про это можно прочитать тут: https://www.ptsecurity.com/ru-ru/products/af/',
                              ' web firewall это межсетевой экрана уровня веб-приложений (web application firewall, WAF)¹, предназначенного для защиты веб-ресурсов от атак из списка OWASP Top 10, DDoS-атак уровня приложений, а также зловредных ботов, часто называется Positive Technologies Application Firewall (PTAF).  Подробнее про это можно прочитать тут: https://www.ptsecurity.com/ru-ru/products/af/',
                              ' Антивирус это антивирусное средство производства лаборатории касперского, состоит из сервера и агентских частей. Подробнее про это можно прочитать тут: https://www.kaspersky.ru/antivirus',
                              ' KSC это антивирусное средство производства лаборатории касперского, состоит из сервера и агентских частей. Подробнее про это можно прочитать тут: https://www.kaspersky.ru/antivirus',
                              ' Касперский это антивирусное средство производства лаборатории касперского, состоит из сервера и агентских частей. Подробнее про это можно прочитать тут: https://www.kaspersky.ru/antivirus',
                              '  Kaspersky это антивирусное средство производства лаборатории касперского, состоит из сервера и агентских частей. Подробнее про это можно прочитать тут: https://www.kaspersky.ru/antivirus',
                              ' антивирус касперского это антивирусное средство производства лаборатории касперского, состоит из сервера и агентских частей. Подробнее про это можно прочитать тут: https://www.kaspersky.ru/antivirus',
                              ' антивирусная защита это антивирусное средство производства лаборатории касперского, состоит из сервера и агентских частей. Подробнее про это можно прочитать тут: https://www.kaspersky.ru/antivirus',
                              ' АВЗ это антивирусное средство производства лаборатории касперского, состоит из сервера и агентских частей. Подробнее про это можно прочитать тут: https://www.kaspersky.ru/antivirus',
                              ' комплексная система антивирусной защиты это антивирусное средство производства лаборатории касперского, состоит из сервера и агентских частей. Подробнее про это можно прочитать тут: https://www.kaspersky.ru/antivirus',
                              ' КСАЗ это антивирусное средство производства лаборатории касперского, состоит из сервера и агентских частей. Подробнее про это можно прочитать тут: https://www.kaspersky.ru/antivirus',
                              ' NGate это программно-аппаратный модуль криптоПРО, беспечивающий возможность предоставления гранулированного доступа конкретным пользователям или группам пользователей к необходимым ресурсам. Подробнее про это можно прочитать тут: https://www.cryptopro.ru/products/ngate',
                              ' шифрование по ГОСТ это программно-аппаратный модуль NGate от компании криптопро, беспечивающий возможность предоставления гранулированного доступа конкретным пользователям или группам пользователей к необходимым ресурсам. Подробнее про это можно прочитать тут: https://www.cryptopro.ru/products/ngate',
                              ' ГОСТ шифрование это программно-аппаратный модуль NGate от компании криптопро, беспечивающий возможность предоставления гранулированного доступа конкретным пользователям или группам пользователей к необходимым ресурсам. Подробнее про это можно прочитать тут: https://www.cryptopro.ru/products/ngate',
                              ' НГейт это программно-аппаратный модуль NGate от компании криптопро, беспечивающий возможность предоставления гранулированного доступа конкретным пользователям или группам пользователей к необходимым ресурсам. Подробнее про это можно прочитать тут: https://www.cryptopro.ru/products/ngate',
                              ' TLS-ГОСТ это программно-аппаратный модуль NGate от компании криптопро, беспечивающий возможность предоставления гранулированного доступа конкретным пользователям или группам пользователей к необходимым ресурсам. Подробнее про это можно прочитать тут: https://www.cryptopro.ru/products/ngate',
                              ' Hashicorp Vault это программное обеспечение hashicorp vault, предназначенное для хранения и управления секретами (пароли, ключи и т.п.). Подробнее про это можно прочитать тут: https://www.hashicorp.com/products/vault',
                              ' Vault это программное обеспечение hashicorp vault, предназначенное для хранения и управления секретами (пароли, ключи и т.п.). Подробнее про это можно прочитать тут: https://www.hashicorp.com/products/vault',
                              ' Вольт это программное обеспечение hashicorp vault, предназначенное для хранения и управления секретами (пароли, ключи и т.п.). Подробнее про это можно прочитать тут: https://www.hashicorp.com/products/vault',
                              ' Хашикорп вольт это программное обеспечение hashicorp vault, предназначенное для хранения и управления секретами (пароли, ключи и т.п.). Подробнее про это можно прочитать тут: https://www.hashicorp.com/products/vault',
                              ' менеджер секретов это программное обеспечение hashicorp vault, предназначенное для хранения и управления секретами (пароли, ключи и т.п.). Подробнее про это можно прочитать тут: https://www.hashicorp.com/products/vault',
                              ' система хранения секретов это программное обеспечение hashicorp vault, предназначенное для хранения и управления секретами (пароли, ключи и т.п.). Подробнее про это можно прочитать тут: https://www.hashicorp.com/products/vault',
                              ' система управления секретами это программное обеспечение hashicorp vault, предназначенное для хранения и управления секретами (пароли, ключи и т.п.). Подробнее про это можно прочитать тут: https://www.hashicorp.com/products/vault',
                              ' КриптоПро это библиотека ядра операционной системы, реализующая шифрование по гост',
                              ' CryptoPro это библиотека ядра операционной системы, реализующая шифрование по гост',
                              ' CSP это библиотека ядра операционной системы, реализующая шифрование по гост',
                              ' криптопровайдер это библиотека ядра операционной системы, реализующая шифрование по гост',
                              ' СТП это служба технической поддержки',
                              ' ТП это служба технической поддержки',
                              ' техподдержка это служба технической поддержки',
                              ' техническая поддержка это служба технической поддержки',
                              ' служба технической поддержки это служба технической поддержки',
                              ' Центр обеспечения безопасности это люди, процессы, технологии и инструменты  для мониторинга информационной  безопасности и реагирования на инциденты.',
                              ' Центр обеспечения информационной безопасности Security Operations Center это люди, процессы, технологии и инструменты  для мониторинга информационной  безопасности и реагирования на инциденты.',
                              ' SOC это люди, процессы, технологии и инструменты  для мониторинга информационной  безопасности и реагирования на инциденты.',
                              ' СОК это люди, процессы, технологии и инструменты  для мониторинга информационной  безопасности и реагирования на инциденты.',
                              ' NTP это передача информации о точном значении времени',
                              ' сервис точного времени это передача информации о точном значении времени',
                              ' сервис времени это передача информации о точном значении времени',
                              ' Network Time Protocol это передача информации о точном значении времени',
                              ' СЗИ это комплекс организационных и технических мер, направленных на обеспечение информационной безопасности',
                              ' система защиты информации это комплекс организационных и технических мер, направленных на обеспечение информационной безопасности',
                              ' СрЗИ это специализированные программные, программно-аппаратные средства, предназначенные для защиты от актуальных угроз',
                              ' средство защиты информации это специализированные программные, программно-аппаратные средства, предназначенные для защиты от актуальных угроз',
                              ' СКЗИ это специализированные программные, программно-аппаратные средства, осуществляющих функции шифрования и генерации электронной подписи (эп)',
                              ' средство криптозащиты это специализированные программные, программно-аппаратные средства, осуществляющих функции шифрования и генерации электронной подписи (эп)',
                              ' средство криптографической защиты это специализированные программные, программно-аппаратные средства, осуществляющих функции шифрования и генерации электронной подписи (эп)',
                              ' ЭП это информация в электронной форме, которая присоединена к другой информации в электронной форме (подписываемой информации) или иным образом связана с такой информацией и которая используется для определения лица, подписывающего информацию ',
                              ' электронная подпись это информация в электронной форме, которая присоединена к другой информации в электронной форме (подписываемой информации) или иным образом связана с такой информацией и которая используется для определения лица, подписывающего информацию ']
        
        # Хранится файл на базу знаний, из которого создаётся RAG
        self.file_path = file_path

        # Загрузка данных из файла
        data_list, sections = self.get_data(self.file_path)
        self.data_list = data_list + self.aditional_data
        self.sections = sections + self.aditional_data

    def load_context(self, context) -> None:
        """
        Загрузка моделей из S3 хранилища.
        """
        from transformers import AutoTokenizer, AutoModel
        
        # Создания экземпляра для взаимодействия с хранилищем S3
        self.s3_provider = S3_provider()
        
        # Создание экземпляра класса Ner модели
        self.ner = self.NerExtractor()

        # Загрузка модели
        self.nlu_model = self.s3_provider.download_from_s3(s3_folder='prod/sber_large_mt_nlu_ru', local_folder='sber_large_mt_nlu_ru')

        print('Loading retriver model...')
        # Загрузка токенизатора для предварительно обученной модели NLU
        self.retriver_tokenizer = AutoTokenizer.from_pretrained(self.nlu_model)

        # Загрузка предварительно обученной модели NLU с переносом на GPU для ускорения обработки
        self.retriver_model = AutoModel.from_pretrained(self.nlu_model).to(device='cuda')
        print('Retriver model loaded')
        
        # Вычисление векторных представлений для списка данных
        self.sentence_embeddings = self.get_embenddings(self.sections, max_length=12).to(device='cuda')
        
    def predict(self, context, model_input: pd.DataFrame) -> str:
        """
        Предсказывает контекст и извлекает сущности, связанные с входным запросом.
        
        Description:
            Этот метод обрабатывает входной запрос (query), используя встроенные функции NLU
            для определения контекста и извлечения соответствующих сущностей. Он возвращает 
            контекст и список сущностей, связанных с запросом.
        Args:
            model_input (pd.DataFrame): DataFrame содержащий id и query.
            user_id (int): id пользователя.
            query (str): Текст запроса, который необходимо обработать.
        Returns:
            str: Строку, содержащий контекст (context) и список сущностей (essence).
        """
        import pandas as pd
        
        # Предобработка DataFrame
        row = model_input.iloc[0]

        user_id = row['id']
        query   = row['query']

        # Контекст
        context = self.get_context(query)
        
        # Сущности
        essence = self.ner.predict(context, query)
        
        query_context = context + " " + essence
        
        res = (pd.DataFrame({'user_id': [user_id], 'query_context': [query_context]})).to_json()
        
        return res

    def get_data(self, file_path: str) -> tuple:
        """
        Получение данных из файла.
        
        Description:
            Функция считывает данные из указанного файла и организует их в удобный для дальнейшей
            обработки формат. Разделяет данные на отдельные секции для упрощения доступа.
        Args:
            file_path (str): Путь к файлу, из которого необходимо извлечь данные.
        Returns:
            tuple: Возвращает кортеж из двух элементов - список всех данных и список секций.
        """
        import re
        import docx
        
        # Загрузка документа
        doc = docx.Document(file_path)

        sections = []
        all_data = []
        data_str = ' '
        block_name = None
        old_sec = None
        link = ' '

        # Регулярное выражение для извлечения ссылки и названия секции
        link_pattern = re.compile(r'\((https?://[^\s]+)\)')
        section_pattern = re.compile(r'«(.*?)»')

        # Итерируем по параграфам документа
        for paragraph in doc.paragraphs:
            text = paragraph.text.strip()

            if 'Страница' in text:
                # Обработка названия секции
                sec_match = section_pattern.search(text)
                sec = sec_match.group(1) if sec_match else text

                # Обработка данных предыдущего блока
                if data_str.strip() and block_name:
                    if '\tПДн' in data_str and len(data_str) > 1000:
                        list_str = []
                    elif '\nРаздел в разработке' in data_str:
                        data_str = ' '
                        list_str = []
                    else:
                        list_str = [data_str[2:] + ' Подробнее про это можно прочитать тут: ' + link]

                    if list_str:
                        all_data += list_str
                        data_str = ' '
                        sections.append(old_sec)

                old_sec = sec
                block_name = text

                # Поиск ссылки в тексте
                link_match = link_pattern.search(text)
                link = link_match.group(1) if link_match else None
                continue

            data_str += '\n' + text

        # Добавление последнего блока данных
        if data_str.strip():
            list_str = [data_str + ' Подробнее про это можно прочитать тут: ' + link]
            all_data += list_str

        return all_data[:-1], sections

    def search_in_context(self, query, sentence_embeddings, model, tokenizer, data_list, treshold, top_k=5) -> list:
        """
        Поиск в контексте с использованием векторных представлений.

        Description:
            Функция выполняет поиск по векторным представлениям, используя косинусное сходство,
            для определения наиболее релевантных предложений из списка данных. Она ищет предложения,
            векторные представления которых находятся на наибольшем косинусном расстоянии от представления запроса,
            превышающем заданный порог сходства.
        Args:
            query (str): Текст запроса.
            sentence_embeddings (Tensor): Тензор векторных представлений предложений.
            model (PreTrainedModel): Предобученная модель для получения векторных представлений.
            tokenizer (Tokenizer): Токенизатор для предобработки текста.
            data_list (list): Список предложений для поиска.
            threshold (float): Пороговое значение сходства для отбора результатов.
            top_k (int, optional): Количество наиболее релевантных предложений для возврата. По умолчанию равно 5.
        Returns:
            list: Список предложений, наиболее релевантных запросу.
        """
        from sentence_transformers import util

        # Получите векторные представления для запроса
        query_embedding = self.get_embenddings(query, max_length=12)

        # Используйте косинусное сходство для поиска наиболее похожих контекстов
        similarities = util.pytorch_cos_sim(query_embedding, sentence_embeddings)[0]

        # Получите индексы наиболее похожих контекстов
        top_k_indices = similarities.argsort(descending=True)[:top_k]

        # Получаем оценки похожести
        similarity_scores = [similarities[i].item() for i in top_k_indices]

        # Получаем наиболее похожие контексты
        results = [data_list[i] for j, i in enumerate(top_k_indices) if similarity_scores[j] > treshold]

        return results

    def search_in_context_with_score(self, query, sentence_embeddings, model, tokenizer, data_list, treshold, top_k=5) -> list:
        """
        Поиск в контексте с оценками сходства.

        Description:
            Функция выполняет поиск по векторным представлениям с возвращением оценок сходства,
            позволяя оценить степень релевантности каждого из результатов поиска. В отличие от функции search_in_context,
            возвращает не только наиболее релевантные предложения, но и их сходство с запросом.
        Args:
            query (str): Текст запроса.
            sentence_embeddings (Tensor): Тензор векторных представлений предложений.
            model (PreTrainedModel): Предобученная модель для получения векторных представлений.
            tokenizer (Tokenizer): Токенизатор для предобработки текста.
            data_list (list): Список предложений для поиска.
            threshold (float): Пороговое значение сходства для отбора результатов.
            top_k (int, optional): Количество наиболее релевантных предложений для возврата. По умолчанию равно 5.
        Returns:
            list of tuples: Список кортежей, где каждый кортеж содержит предложение и его оценку сходства с запросом.
        """
        from sentence_transformers import util

        # Получаем векторные представления для запроса
        query_embedding = self.get_embenddings(query, max_length=12)

        # Используем косинусное сходство для поиска наиболее похожих контекстов
        similarities = util.pytorch_cos_sim(query_embedding, sentence_embeddings)[0]

        # Получаем индексы наиболее похожих контекстов
        top_k_indices = similarities.argsort(descending=True)[:top_k]

        # Получаем оценки похожести
        similarity_scores = [similarities[i].item() for i in top_k_indices]

        # Получаем наиболее похожие контексты
        results = [(data_list[i],similarity_scores[j]) for j, i in enumerate(top_k_indices) if similarity_scores[j] > treshold]

        return results

    def get_context(self, question: str) -> str:
        """
        Description:
            Эта функция использует список синонимов для расширения поиска, пытаясь найти
            наиболее подходящий контекстный ответ на вопрос.
        Args:
            question (str): Вопрос для обработки.
        Returns:
            str: Контекстный ответ на вопрос.
        Exceptions:
            str: Возвращает пустую строку, если не найдены соответствующей контекстный ответ на вопрос.
        """

        # Приведение вопроса к нижнему регистру для унификации поиска
        question = question.lower()

        # Инициализация списка для проверки синонимов
        check_list = []

        # Перебор списка синонимов для расширения запроса
        for synonyms in self.synonyms_dicts:
            for synonym in synonyms:
                if synonym.lower() in question:
                    check_list += [question.replace(synonym.lower(), el) for el in synonyms]

        # Список для контекстных ответов
        context_responses = []

        # Ищем контекстные ответы для всех вариаций вопроса
        if check_list:
            for quest in check_list:
                context_responses += [el[0] for el in sorted(
                    self.search_in_context_with_score(quest, self.sentence_embeddings, self.retriver_model, self.retriver_tokenizer, self.data_list, 0.22, top_k=5),
                    reverse=True, key=lambda x: x[1])]
        else:
            context_responses = self.search_in_context(question, self.sentence_embeddings, self.retriver_model, self.retriver_tokenizer, self.data_list, 0.22, top_k=5)

        # Возвращает контекст
        return '\n\n'.join(context_responses[:3]) if context_responses else ' '

    def extract_named_entities(question) -> str:
        """
        Извлекает именованные сущности из вопроса, классифицирует их, и возвращает соответствующий контент.
                
        Description:
            Эта функция использует NER (Named Entity Recognition) для идентификации именованных сущностей в заданном вопросе.
            На основе этих сущностей функция ищет соответствующие классы и подклассы в данных, и возвращает связанный с ними контент.
        Args:
            question (str): Вопрос для обработки.
        Returns:
            str: Строка с контентом, соответствующим именованным сущностям вопроса, и ссылкой для дополнительной информации.
        Exceptions:
            str: Возвращает пустую строку, если не найдены соответствующие именованные сущности или классы.
        """
        from langchain.vectorstores import Chroma
        from langchain.docstore.document import Document
        
        # NER
        # Получение именнованых сущностей
        entries = self.extractor.get_entities(question)
        ent = [question[el[-2]:el[-1]] for el in entries]

        if ent != []:
            # Получение классов с именоваными сущностями
            cl_s = {el[1]:el[0].page_content  for el in cl_s if ent[0] in el[0].page_content}
        elif cl_s == []:
            return ' '

        else:
            tp_class = sorted(cl_s, key=lambda x: x[1])[0][0].page_content
            test = tp_class

        if isinstance(cl_s, dict):
            if list(cl_s.keys()) != []:
                # print(clas.keys())
                tp_class = cl_s[min(cl_s.keys())]

                # Получение подклассов
                subclasses = Chroma.from_documents([Document(page_content=el) for el in data[tp_class].keys()], embeddings, collection_name=''.join(random.choice(characters) for _ in range(10)))

                # Получение ближайших под_классов
                tp_subclass = subclasses.similarity_search(question, k = 3)
                return '\n\n'.join([data[tp_class][el.page_content] for el in tp_subclass])  + ' Подробнее прочитать об этом вы можете тут: ' + tp_class
        else:
            # Получение подклассов
            subclasses = Chroma.from_documents([Document(page_content=el) for el in data[tp_class].keys()], embeddings, collection_name=''.join(random.choice(characters) for _ in range(10)))

            # Получение ближайших под_классов
            tp_subclass = subclasses.similarity_search(question, k = 3)
            test = tp_subclass
            return '\n\n'.join([data[tp_class][el.page_content] for el in tp_subclass]) + ' Подробнее прочитать об этом вы можете тут: ' + tp_class


    def get_entities(self, text: str) -> list:
        """
        Description:
            Извлекает сущности из предоставленного текста.
        Args:
            text (str): Текст для извлечения сущностей.
        Returns:
            list: Список извлеченных сущностей.
        """
        return self.extractor.get_entities(text)

    def get_embenddings(self, data_list, max_length=12) -> torch.Tensor:
        """
        Description:
            Вычисляет векторные представления для списка данных.
        Args:
            data_list (list): Список данных для обработки.
            max_length (int, optional): Максимальная длина вектора. По умолчанию 12.
        Returns:
            torch.Tensor: Тензор векторных представлений.
        """
        import torch
        
        # Токенизация входного списка данных с заданными параметрами.
        # padding=True обеспечивает одинаковую длину всех последовательностей.
        # truncation=True усекает данные, превышающие max_length.
        # return_tensors='pt' возвращает тензоры PyTorch.
        try:
            encoded_input = self.retriver_tokenizer(data_list, padding=True, truncation=True, max_length=64, return_tensors='pt')

            # Перемещение данных на GPU для ускорения вычислений (если доступно).
            encoded_input = {key: value.to('cuda') for key, value in encoded_input.items()}

            # Вычисление векторных представлений без обучения модели (no_grad).
            with torch.no_grad():
                model_output = self.retriver_model(**encoded_input)

            # Вызов функции mean_pooling для получения усредненных векторных представлений.
            return self.mean_pooling(model_output, encoded_input['attention_mask'])
        except Exception as e:
            print(f"Ошибка при получении векторных представлений: {e}")

    def mean_pooling(self, model_output, attention_mask) -> torch.Tensor:
        """
        Выполняет усреднение пулинга для токенов.

        Description:
            Эта функция используется для агрегирования выходных данных модели (представлений токенов)
            в одно усредненное векторное представление для каждого входного примера.
        Args:
            model_output: Выходные данные модели.
            attention_mask: Маска внимания.
        Returns:
            torch.Tensor: Усредненное векторное представление.
        """
        import torch
        
        # Получение векторных представлений токенов из последнего скрытого состояния модели
        token_embeddings = model_output.last_hidden_state

        # Расширение маски внимания для соответствия размерам токенных векторов
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.shape)

        # Умножение каждого токенного вектора на его маску внимания и суммирование
        # для получения общего векторного представления для каждого примера
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)

        # Подсчет количества токенов для каждого примера (с учетом маски внимания)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)

        # Деление суммы векторов на количество токенов для получения усредненного представления
        return sum_embeddings / sum_mask
    
    class NerExtractor(mlflow.pyfunc.PythonModel):
        """
        Description:
            Класс для извлечения именованных сущностей (NER) из текста, использующий предобученные модели из библиотеки transformers.
        Args:
            token_pred_pipeline (pipeline): Пайплайн для классификации токенов, используемый для извлечения сущностей.
        Functions:
            concat_entities: Объединяет именованные сущности, принадлежащие к одному и тому же типу и расположенные рядом друг с другом.
            get_entities: Извлекает именованные сущности из предоставленного текста.
        """
        def __init__(self):
            """
            Инициализирует экстрактор сущностей с использованием указанной модели.
            """
            # Загрузка модели
            self.load_context(context = None)

        def load_context(self, context) -> None:
            """
            Загрузка моделей из S3 хранилища.
            """
            from transformers import pipeline, AutoModelForTokenClassification
            
            # Создания экземпляра для взаимодействия с хранилищем S3
            self.s3_provider = S3_provider()
        
            # Загрузка модели
            model_checkpoint = self.ner_model = self.s3_provider.download_from_s3(s3_folder='prod/vicuna_bot/LaBSE_ner_nerel', local_folder='LaBSE_ner_nerel')
            
            # Инициализация pipeline для классификации токенов на GPU, используя агрегацию средним значением
            self.token_pred_pipeline = pipeline("token-classification", 
                                                model=model_checkpoint, 
                                                aggregation_strategy="average",
                                                device='cuda'
                                               )


        def predict(self, context, query) -> list:
            """
            Description:
                Прогнозирует и извлекает именованные сущности из предоставленного текста.
            Args:
                query (str): Текстовый запрос, из которого необходимо извлечь именованные сущности.
            Returns:
                list: Список извлеченных именованных сущностей. Каждая сущность представлена в виде кортежа (тип сущности, текст сущности).
            """
            essence = self.get_entities(query) 
            
            return essence

        @staticmethod
        def concat_entities(entities) -> list:
            """
            Description:
                Объединяет последовательные именованные сущности одного типа в одну сущность.
            Args:
                entities (list of tuples): Список кортежей именованных сущностей.
            Returns:
                list: Объединенный список именованных сущностей.
            """
            if not entities:
                return []

            # Инициализация списка для объединенных сущностей
            concat_entities = []
            prev_entity_type, prev_entity_text = entities[0]

            for entity_type, entity_text in entities[1:]:
                if entity_type == prev_entity_type:
                    prev_entity_text += " " + entity_text
                else:
                    concat_entities.append((prev_entity_type, prev_entity_text))
                    prev_entity_type, prev_entity_text = entity_type, entity_text

            # Добавление последней сущности
            concat_entities.append((prev_entity_type, prev_entity_text))

            return concat_entities

        def get_entities(self, text: str) -> str:
            """
            Description:
                Извлекает именованные сущности из текста и возвращает их в виде строки.
            Args:
                text (str): Текст для извлечения сущностей.
            Returns:
                str: Строка, содержащая именованные сущности и их типы, разделенные запятыми.
            Error:
                AssertionError: Если текст пустой.
            """
            # Проверка, что текст не пустой
            assert len(text) > 0, "Предоставленный текст пустой."
        
            entities = self.token_pred_pipeline(text)

            # Преобразование списка объединенных сущностей в строку
            concatenated_entities = self.concat_entities(entities)
            return ', '.join([f'{etype}, {etext}' for etype, etext in concatenated_entities])

### Local test

In [6]:
import pandas as pd
import requests
import json
import base64

# Создаем DataFrame с указанными полями и текстом
data = {
    "id": 611,
    "query": ["Как защититься от DDoS атак?"]
}
responce = {
    "prediction": ["Ответ будет в виде текста"]
}

input_df = pd.DataFrame(data)

output = pd.DataFrame(responce)

In [7]:
# Установка URI для MLflow трекинг сервера
mlflow.set_tracking_uri("http://mlflow")

# # Создание эксперимента
# mlflow.create_experiment('domain_nlu_retriever')

In [9]:
notebook_file = "./Domain_retriever.ipynb"

# Начало MLflow эксперимента
with mlflow.start_run(experiment_id=21):
        mlflow.pyfunc.log_model(
            artifact_path='domain_retriever',
            python_model=Domain_Retriever(),
            signature=mlflow.models.signature.infer_signature(input_df, output),
            artifacts={"log": './log.log'},
            registered_model_name='domainretriever',
            pip_requirements="requirements.txt",)
        
        # Зарегистрировать Jupyter Notebook как артефакт
        mlflow.log_artifact(notebook_file, artifact_path="notebooks")

Registered model 'domainretriever' already exists. Creating a new version of this model...
2024/03/22 12:10:45 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: domainretriever, version 52
Created version '52' of model 'domainretriever'.


In [8]:
logged_model = 'runs:/6d833889c60445949af41b7de9e10eb9/domain_retriever'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

Loading NER model...


/opt/conda/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Retriver NER Loaded
Loading retriver model...
Retriver model loaded


In [9]:
answer = json.loads(loaded_model.predict(pd.DataFrame(data)))

In [10]:
answer

{'user_id': {'0': 611},
 'query_context': {'0': ' антивирусная защита это антивирусное средство производства лаборатории касперского, состоит из сервера и агентских частей. Подробнее про это можно прочитать тут: https://www.kaspersky.ru/antivirus\n\n Антивирус это антивирусное средство производства лаборатории касперского, состоит из сервера и агентских частей. Подробнее про это можно прочитать тут: https://www.kaspersky.ru/antivirus\n\n комплексная система антивирусной защиты это антивирусное средство производства лаборатории касперского, состоит из сервера и агентских частей. Подробнее про это можно прочитать тут: https://www.kaspersky.ru/antivirus EVENT, DDoS атак'}}

In [11]:
answer['query_context']['0']

' антивирусная защита это антивирусное средство производства лаборатории касперского, состоит из сервера и агентских частей. Подробнее про это можно прочитать тут: https://www.kaspersky.ru/antivirus\n\n Антивирус это антивирусное средство производства лаборатории касперского, состоит из сервера и агентских частей. Подробнее про это можно прочитать тут: https://www.kaspersky.ru/antivirus\n\n комплексная система антивирусной защиты это антивирусное средство производства лаборатории касперского, состоит из сервера и агентских частей. Подробнее про это можно прочитать тут: https://www.kaspersky.ru/antivirus EVENT, DDoS атак'

In [10]:
# Завершение MLflow эксперимента
mlflow.end_run()

### Web test

In [11]:
# Создание словаря с данными
data = {
    'id': 42,
    'query': [base64.b64encode("Как защититься от DDoS атак?".encode("utf-8")).decode("utf-8")],
}

df = pd.DataFrame(data)

# Передаем запрос в Domain retriever для определения контекста
model_url = 'https://aiplatform.mos.ru/operation/domainretriever/invocations'

response = json.loads(requests.post(model_url, json={'dataframe_records': df.to_dict(orient='records')}).json()['predictions'])

In [14]:
response

{'user_id': {'0': 42},
 'query_context': {'0': ' Hashicorp Vault это программное обеспечение hashicorp vault, предназначенное для хранения и управления секретами (пароли, ключи и т.п.). Подробнее про это можно прочитать тут: https://www.hashicorp.com/products/vault\n\n Хашикорп вольт это программное обеспечение hashicorp vault, предназначенное для хранения и управления секретами (пароли, ключи и т.п.). Подробнее про это можно прочитать тут: https://www.hashicorp.com/products/vault\n\n web firewall это межсетевой экрана уровня веб-приложений (web application firewall, WAF)¹, предназначенного для защиты веб-ресурсов от атак из списка OWASP Top 10, DDoS-атак уровня приложений, а также зловредных ботов, часто называется Positive Technologies Application Firewall (PTAF).  Подробнее про это можно прочитать тут: https://www.ptsecurity.com/ru-ru/products/af/ '}}

In [12]:
# Обработка ответа
query_context = response['query_context']['0']

In [13]:
query_context

' Hashicorp Vault это программное обеспечение hashicorp vault, предназначенное для хранения и управления секретами (пароли, ключи и т.п.). Подробнее про это можно прочитать тут: https://www.hashicorp.com/products/vault\n\n Хашикорп вольт это программное обеспечение hashicorp vault, предназначенное для хранения и управления секретами (пароли, ключи и т.п.). Подробнее про это можно прочитать тут: https://www.hashicorp.com/products/vault\n\n web firewall это межсетевой экрана уровня веб-приложений (web application firewall, WAF)¹, предназначенного для защиты веб-ресурсов от атак из списка OWASP Top 10, DDoS-атак уровня приложений, а также зловредных ботов, часто называется Positive Technologies Application Firewall (PTAF).  Подробнее про это можно прочитать тут: https://www.ptsecurity.com/ru-ru/products/af/ '